### **Introdução**

O presente documento tem como objetivo principal treinar e avaliar modelos preditivos, visando responder as perguntas: *"Qual será o placar final?", "Qual o Total de Gols da partida?" e "Quem será o vencedor do jogo?"*, do grupo Mestres do Placar, no projeto para a IBM.

O projeto está dividido em várias etapas que incluem a preparação dos dados, criação de modelos de classificação, previsão de vencedores e gols, além da avaliação do desempenho dos modelos usando métricas tradicionais e matrizes de confusão.

- Total de Gols
*   Vencedor do Jogo
*   Gols de Casa e Visitantes


### **0. Análise de diferentes modelos**

Nesta célula, é realizada a análise de diferentes modelos de machine learning, com foco no Random Forest como modelo candidato.



In [ ]:
# Importação das bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Função para avaliar modelos
def avaliar_modelo(modelo, X_train, X_test, y_train, y_test, nome_modelo):
    # Treina o modelo
    modelo.fit(X_train, y_train)
    # Faz as previsões
    y_pred = modelo.predict(X_test)
    # Calcula o RMSE e o R²
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{nome_modelo} - RMSE: {rmse}, R²: {r2}")
    return rmse, r2

# Carregar os dados
df = pd.read_csv('arquivo_codificado.csv')

# Definir as features e os alvos
features = ['home_team_name_mapeado', 'away_team_name_mapeado', 'Home Team Pre-Match xG',
            'Away Team Pre-Match xG', 'home_team_shots_on_target', 'away_team_shots_on_target',
            'minutes_per_goal_scored_overall', 'minutes_per_goal_conceded_overall',
            'average_attendance_home', 'average_attendance_away']

X = df[features]
y_home = df['home_team_goal_count']
y_away = df['away_team_goal_count']

# Dividir os dados em treino e teste
X_train, X_test, y_home_train, y_home_test = train_test_split(X, y_home, test_size=0.2, random_state=42)
_, _, y_away_train, y_away_test = train_test_split(X, y_away, test_size=0.2, random_state=42)


modelos = {
    'RandomForest - Casa': RandomForestRegressor(n_estimators=100, random_state=42),
    'RandomForest - Visitante': RandomForestRegressor(n_estimators=100, random_state=42),
    'LinearRegression - Casa': LinearRegression(),
    'LinearRegression - Visitante': LinearRegression()
}

# Avaliar os modelos
resultados = []
for nome_modelo, modelo in modelos.items():
    if 'Casa' in nome_modelo:
        rmse, r2 = avaliar_modelo(modelo, X_train, X_test, y_home_train, y_home_test, nome_modelo)
    else:
        rmse, r2 = avaliar_modelo(modelo, X_train, X_test, y_away_train, y_away_test, nome_modelo)
    resultados.append({'Modelo': nome_modelo, 'RMSE': rmse, 'R²': r2})

# Criar um DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)
print(df_resultados)


Após a escolha de um modelo para seguir, aqui iremos começar com a importação das ferramentas necessárias e carregamento dos dados.


### **1. Importação das Bibliotecas**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

Nesta célula, importamos bibliotecas essenciais para a análise de dados, divisão de dados de treino/teste, construção do modelo de Random Forest e avaliação de métricas como acurácia, precisão, recall e f1-score.

***Bibliotecas:***

`pandas: `Manipulação e análise de dados.

`train_test_split` da sklearn: Divide os dados em treino e teste.

`RandomForestClassifier da sklearn:` Modelo de aprendizado supervisionado para classificação.

`accuracy_score, precision_score, recall_score, f1_score, confusion_matrix: `Funções para avaliar a performance do modelo.

`matplotlib: `Biblioteca para visualizações gráficas.

`seaborn:` Visualização de dados

### **2. Carregamento e preparação o conjunto de dados**

**Preparação dos Dados**

Os dados utilizados nesse Colab já foram previamente tratados e preparados para a modelagem.

O processo de preparação incluiu as seguintes etapas:

`Limpeza dos Dados: `Remoção de valores ausentes (missing values) e tratamento adequado para colunas com dados faltantes.

`Tratamento de Outliers: `Identificação e tratamento de valores atípicos (outliers) para garantir que eles não prejudiquem a performance dos modelos.

`Codificação (Encoding) de Variáveis Categóricas:` As variáveis categóricas foram convertidas para um formato numérico utilizando técnicas de One-Hot Encoding, como `home_team_name_mapeado` e `away_team_name_mapeado`.


In [ ]:
df = pd.read_csv('arquivo_codificado.csv') # Carrega o conjunto de dados

df['home_team_win'] = (df['home_team_goal_count'] > df['away_team_goal_count']).astype(int)

df.head()

### **3. Definição de Features e Alvos**

Definimos as variáveis independentes (features) usadas para prever o número de gols e a variável-alvo (target) que representa a quantidade total de gols (total_goal_count) e o vencedor (home_team_win).

In [ ]:
# Definir as features e o alvo

features = ['Home Team Pre-Match xG', 'Away Team Pre-Match xG',
             'home_team_shots_on_target', 'away_team_shots_on_target',
             'minutes_per_goal_scored_overall', 'minutes_per_goal_conceded_overall',
             'average_attendance_home', 'average_attendance_away']

# Para previsão do total de gols
target_total_goals = 'total_goal_count'

# Para prever o time vencedor
target_winner = 'home_team_win'

# Separando as features e o alvo para total de gols
X_total_goals = df[features]
y_total_goals = df[target_total_goals]

# Separando as features e o alvo para previsão do vencedor
X_winner = df[features]
y_winner = df[target_winner]


### **4. Divisão do Conjunto de Dados para Treino e Teste**

Depois de preparados, os dados são divididos em conjuntos de treino e teste para evitar overfitting e garantir uma avaliação justa dos modelos. Essa divisão é feita com a função train_test_split da biblioteca sklearn, que permite que 80% dos dados sejam usados para o treinamento e 20% para os testes.

**Parâmetros:**

`test_size=0.2: `20% dos dados são reservados para teste.

`random_state=42: `Garante a reprodutibilidade dos resulta

In [ ]:
# Divide os dados para treino e teste para total de gols
X_train_goals, X_test_goals, y_train_goals, y_test_goals = train_test_split(X_total_goals, y_total_goals, test_size=0.2, random_state=42)


# Divide os dados para treino e teste para previsão do vencedor
X_train_winner, X_test_winner, y_train_winner, y_test_winner = train_test_split(X_winner, y_winner, test_size=0.2, random_state=42)


### **5. Treinamento do Modelo (Random Forest) para Total de Gols**

Um modelo de Random Forest é instanciado com 1000 árvores de decisão e treinado usando o conjunto de dados de treino para prever o total de gols.

**Parâmetros:**

`n_estimators=1000`: Define o número de árvores na floresta.

`class_weight='balanced'`: Ajusta o peso das classes para lidar com classes desbalanceadas.

In [ ]:
# Usando modelo Random Forest para total de gols
model_goals = RandomForestClassifier(n_estimators=1000, random_state=42, class_weight='balanced')

# Treina o modelo com os dados de treinamento
model_goals.fit(X_train_goals, y_train_goals)


### **6.1. Modelo para previsão para Total de Gols**

Essa parte do código foca em gerar as previsões do modelo para o total de gols, avaliar o desempenho dessas previsões e visualizar os resultados utilizando uma matriz de confusão.

Após a previsão, o modelo é avaliado com métricas como acurácia, precisão, recall e F1-score.

**Métricas utilizadas:**

`accuracy_score: `calcula a proporção de previsões corretas feitas pelo modelo em relação ao total de previsões feitas.

`precision_score: `calcula a proporção de verdadeiros positivos em relação ao total de positivos previstos.

`recall_score: `calcula a proporção de verdadeiros positivos em relação ao total de positivos reais.

`f1_score:` média harmônica da precisão e do recall, proporcionando um equilíbrio entre os dois.


In [ ]:
# Faz as previsões no conjunto de teste
y_pred_goals = model_goals.predict(X_test_goals)

# Avaliação do modelo para total de gols
print(f'Acurácia (Total de Gols): {accuracy_score(y_test_goals, y_pred_goals):.2f}')
print(f'Precisão (Total de Gols): {precision_score(y_test_goals, y_pred_goals, average="weighted"):.2f}')
print(f'Recall (Total de Gols): {recall_score(y_test_goals, y_pred_goals, average="weighted"):.2f}')
print(f'F1-Score (Total de Gols): {f1_score(y_test_goals, y_pred_goals, average="weighted"):.2f}')


**Matriz de Confusão para Previsão do Total de Gols**

A matriz de confusão é gerada para avaliar a performance do modelo na previsão de número de gols.

Isso é feito usando a função `confusion_matrix`.

In [ ]:
# Matriz de confusão para o total de gols
cm_goals = confusion_matrix(y_test_goals, y_pred_goals)

# Exibir a matriz de confusão usando seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm_goals, annot=True, fmt='d', cmap='Blues', xticklabels=True, yticklabels=True)
plt.title('Matriz de Confusão - Total de Gols')
plt.xlabel('Previsão')
plt.ylabel('Real')
plt.show()


### **6.2. Modelo para previsão do vencedor do jogo**

`RandomForestClassifier:` Cria e treina um modelo Random Forest para prever o vencedor do jogo.

In [ ]:
# Modelo Random Forest para previsão do vencedor
model_winner = RandomForestClassifier(n_estimators=1000, random_state=42, class_weight='balanced')

# Treina o modelo com os dados de treinamento
model_winner.fit(X_train_winner, y_train_winner)

# Faz previsões no conjunto de teste
y_pred_winner = model_winner.predict(X_test_winner)

# Avaliação do modelo para previsão do vencedor
print(f'Acurácia (Vencedor do Jogo): {accuracy_score(y_test_winner, y_pred_winner):.2f}')
print(f'Precisão (Vencedor do Jogo): {precision_score(y_test_winner, y_pred_winner, average="weighted"):.2f}')
print(f'Recall (Vencedor do Jogo): {recall_score(y_test_winner, y_pred_winner, average="weighted"):.2f}')
print(f'F1-Score (Vencedor do Jogo): {f1_score(y_test_winner, y_pred_winner, average="weighted"):.2f}')


**Matriz de Confusão para previsão do vencedor**

Gera e exibe a matriz de confusão para o modelo de previsão do vencedor, utilizando as funções `confusion_matrix e ConfusionMatrixDisplay `.


In [ ]:
# Matriz de confusão para previsão do vencedor
cm_winner = confusion_matrix(y_test_winner, y_pred_winner)
disp_winner = ConfusionMatrixDisplay(confusion_matrix=cm_winner)
disp_winner.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusão - Vencedor do Jogo")
plt.show()

### **6.3. Modelo para previsão da quantidade de gols para cada time, da casa e visitantes**

Nesta parte do código, previsões são feitas para os gols da equipe da casa e gols da equipe visitante usando o modelo `model_goals`. A partir dessas previsões, o código calcula os resultados previstos (vitória da casa, vitória do visitante, ou empate) e os compara com os resultados reais.

In [ ]:
y_pred_home_goals = model_goals.predict(X_test_goals)  # Previsão para os gols da casa
y_pred_away_goals = model_goals.predict(X_test_goals)  # Previsão para os gols dos visitantes

In [ ]:
# Define os resultados reais e previstos
y_pred_results = []
y_true_results = []


# Define os resultados reais e previstos para cada jogo
for i in range(len(y_test_goals)):
    home_team_goals_real = df['home_team_goal_count'].iloc[i]
    away_team_goals_real = df['away_team_goal_count'].iloc[i]

    # Resultado real
    if home_team_goals_real > away_team_goals_real:
        y_true_results.append('home_win')
    elif home_team_goals_real < away_team_goals_real:
        y_true_results.append('away_win')
    else:
        y_true_results.append('draw')

    # Resultado previsto
    if y_pred_home_goals[i] > y_pred_away_goals[i]:
        y_pred_results.append('home_win')
    elif y_pred_home_goals[i] < y_pred_away_goals[i]:
        y_pred_results.append('away_win')
    else:
        y_pred_results.append('draw')

print(f'Acurácia (Resultado do Jogo): {accuracy_score(y_true_results, y_pred_results):.2f}')
print(f'Precisão (Resultado do Jogo): {precision_score(y_true_results, y_pred_results, average="weighted"):.2f}')
print(f'Recall (Resultado do Jogo): {recall_score(y_true_results, y_pred_results, average="weighted"):.2f}')
print(f'F1-Score (Resultado do Jogo): {f1_score(y_true_results, y_pred_results, average="weighted"):.2f}')
